In [1]:
# Rotating num_cat catalogs num_rot times
# hume, based on Rachel's code

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf 
from sklearn.preprocessing import StandardScaler
import os
import re
from scipy.spatial.transform import Rotation as R
import time

#time
from time import process_time
from time import time

start_process = process_time()
start_time = time()

tf.random.set_seed(982369253487)

In [3]:
# how many catalogs to rotate

num_cat = 300
num_rot = 100

# In the path below create a directory called rotated_{num_cat}_{num_rot}

# filenames and paths

filepathin = f'C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/'

filepathout = f'C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotations using Rachels code/'

filenameBF = f'C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/true_bulkflows150.npy'

print(filepathin)
print(filepathout)
print(filenameBF)


C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/
C:/Users/yaras/Documents/Research/Feldman/rotated-outerrim-cz-rand/rotations using Rachels code/
C:/Users/yaras/Documents/Research/Feldman/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/true_bulkflows150.npy


In [4]:
files_list = [] 
j = 0
k = 0
for x in os.listdir(filepathin):

    if x[0] == str('C'): 
        j =+ 1
        files_list.append(x)
    else:
        k += 1
        pass
print(j, k)

1 9


In [5]:
#scale of bulkflows used for this set is 150 MPC 
# bulkflows = np.load("/content/drive/My Drive/Research/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/true_bulkflows150.npy")

bulkflows = np.load(filenameBF)
bulkflows[:,0]

array([ -25.39076889,  148.71427005, -144.27017343])

In [6]:
# if only some of the catalogs rotate num_rot times

galaxy_index = []
path = filepathin

k = 0
while k < num_cat:    
    
    x = files_list[k]

    data = np.load(path + x)
    time1 = process_time()

    i = 0 
    while i < num_rot: 
        time2 = process_time()

        temp = re.findall(r'\d+', x)
        res = list(map(int, temp))
        bulkflow_ind = res[2]
        galaxy_index.append(bulkflow_ind)

        bulkflow = bulkflows[:,bulkflow_ind] #0 - bulkflow ind

        rshift = data[:,6]
        distmod = 5*np.log(data[:,7]) + 25 
        distmod_err = np.random.normal(0, 0.4, len(distmod))

        glon= data[:,10]*np.pi/180
        glat= data[:,11]*np.pi/180

        #make glon and glat into array of positions on the unit sphere
        pos = np.array([np.cos(glon)*np.cos(glat),np.sin(glon)*np.cos(glat),np.sin(glat)])
        pos = np.transpose(pos)

        alpha = 2*np.pi*np.random.random()  #generate random rotation angles
        beta = np.pi*np.random.random()
        gamma = 2*np.pi*np.random.random()
        
        r = R.from_euler('xzx', [alpha,beta,gamma]) #create the rotation

        posprime = r.apply(pos)  #apply the rotation to the vectors

        #bulkflows already in cartesian, so no transformation like above? 
        bulkflowprime = r.apply(bulkflow)

        #rotated_bulkflows.append(bulkflowprime)

        glonprime = np.arctan2(posprime[:,1], posprime[:,0]) * 180 / np.pi  #convert back to glon and glat
        glonprime[glonprime < 0] = 360 + glonprime[glonprime < 0]
        
        glatprime = np.arcsin(posprime[:,2]) * 180 / np.pi

            # add rshift, dist mod, dist mod error, glon, glat, bulk flow to new file 
        #    with open('/content/drive/My Drive/Rotated/Rotated Catalog' + str(bulkflow_ind) + '_' + str(i) + '.npy.npz', 'wb') as f:
                #stuff = np.array([rshift,glonprime,glatprime,distmod,distmod_err,bulkflowprime])
        #        np.savez(f, redshift=rshift, glon=glonprime, glat=glatprime, distm = distmod, distmerr = distmod_err, bulkflow=bulkflowprime)

        file_label = str(bulkflow_ind) + '_' + str(i) + '.npz'
        with open(filepathout + file_label, 'wb') as f:
            np.savez(f,
                     redshift=rshift, 
                     glon=glonprime, 
                     glat=glatprime,
                     distm = distmod, 
                     distmerr = distmod_err,
                     bulkflow=bulkflowprime)
            f.close()

        i += 1

    k += 1
                    


In [7]:
print("\n%6d rotations on %4d catalogs in %6.2f s %6.2f s" % 
      (num_rot,num_cat,
       process_time() - start_process,
       time() - start_time), end = '\r')

#files_list[:num_cat] #didn't work???